# GeoNet Seismicity For the Taupo Area, Focussed on the Four Geothermal Fields Used For Power Production

This is hopefully a useful tool should we get enquiries about induced seismicity. Data is GeoNet so can display freely.

**Import modules, etc**

In [ ]:
#import cartopy
import cartopy.crs as ccrs
import cartopy.io.img_tiles as cimgt
from cartopy.io.img_tiles import OSM

import pandas as pd
import numpy as np
import pyproj
from math import floor

import datetime

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.transforms import offset_copy
from matplotlib import patheffects
import matplotlib.gridspec as gridspec
import warnings; warnings.simplefilter('ignore')

%matplotlib inline

In [ ]:
plt.style.use('fivethirtyeight')

In [ ]:
#full width notebook display
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

## Functions

In [ ]:
def haversine_np(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)

    All args must be of equal length.    

    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
    return km

def utm_from_lon(lon):
    """
    utm_from_lon - UTM zone for a longitude

    Not right for some polar regions (Norway, Svalbard, Antartica)

    :param float lon: longitude
    :return: UTM zone number
    :rtype: int
    """
    return floor( ( lon + 180 ) / 6) + 1

def scale_bar(ax, proj, length, location=(0.5, 0.05), linewidth=3,
              units='km', m_per_unit=1000):
    """

    http://stackoverflow.com/a/35705477/1072212
    ax is the axes to draw the scalebar on.
    proj is the projection the axes are in
    location is center of the scalebar in axis coordinates ie. 0.5 is the middle of the plot
    length is the length of the scalebar in km.
    linewidth is the thickness of the scalebar.
    units is the name of the unit
    m_per_unit is the number of meters in a unit
    """
    # find lat/lon center to find best UTM zone
    x0, x1, y0, y1 = ax.get_extent(proj.as_geodetic())
    # Projection in metres
    utm = ccrs.UTM(utm_from_lon((x0+x1)/2))
    # Get the extent of the plotted area in coordinates in metres
    x0, x1, y0, y1 = ax.get_extent(utm)
    # Turn the specified scalebar location into coordinates in metres
    sbcx, sbcy = x0 + (x1 - x0) * location[0], y0 + (y1 - y0) * location[1]
    # Generate the x coordinate for the ends of the scalebar
    bar_xs = [sbcx - length * m_per_unit/2, sbcx + length * m_per_unit/2]
    # buffer for scalebar
    buffer = [patheffects.withStroke(linewidth=5, foreground="w")]
    # Plot the scalebar with buffer
    ax.plot(bar_xs, [sbcy, sbcy], transform=utm, color='k',
        linewidth=linewidth, path_effects=buffer)
    # buffer for text
    buffer = [patheffects.withStroke(linewidth=3, foreground="w")]
    # Plot the scalebar label
    t0 = ax.text(sbcx, sbcy, str(length) + ' ' + units, transform=utm,
        horizontalalignment='center', verticalalignment='bottom',
        path_effects=buffer, zorder=2)
    left = x0+(x1-x0)*0.05
    # Plot the N arrow
    t1 = ax.text(left, sbcy, u'\u25B2\nN', transform=utm,
        horizontalalignment='center', verticalalignment='bottom',
        path_effects=buffer, zorder=2)
    # Plot the scalebar without buffer, in case covered by text buffer
    ax.plot(bar_xs, [sbcy, sbcy], transform=utm, color='k',
        linewidth=linewidth, zorder=3)
    
# or to use m instead of km
# scale_bar(ax, ccrs.Mercator(), 100000, m_per_unit=1, units='m')
# or to use miles instead of km
# scale_bar(ax, ccrs.Mercator(), 60, m_per_unit=1609.34, units='miles')

#need specific font to show triangle for north arrow
mpl.rc('font', family='DejaVu Sans')

## Set Up Parameters

In [ ]:
start = '2008-01-01T00:00:00'
#no end is used, so search defaults to now

In [ ]:
ngatamariki_latref = -38.546143
ngatamariki_lonref = 176.195504

rotokawa_latref = -38.615574
rotokawa_lonref = 176.215

mokai_latref = -38.519
mokai_lonref = 175.918

wairakei_tauhara_latref = -38.6488
wairakei_tauhara_lonref = 176.0890

In [ ]:
#box for search area
latmin = -38.75
latmax = -38.48
lonmin = 175.84
lonmax = 176.27
bbox = '175.84,-38.75,176.27,-38.48'
maxdep = '20'

url = 'https://quakesearch.geonet.org.nz/csv?bbox='+bbox+'&maxdepth='+maxdep+'&startdate='+start

In [ ]:
eqs = pd.read_csv(url, parse_dates=['origintime'], index_col='origintime', usecols=[2,4,5,6,7])

In [ ]:
eqs.tail()

## Plot hypocentres for the complete area on Open Street Map basemap

In [ ]:
#imagery
imagery = OSM()
ax = plt.axes(projection=imagery.crs)
ax.set_extent([lonmin, lonmax, latmin, latmax])
ax.add_image(imagery, 10)

#hypocentres, symbol size=magnitude squared, as scatter symbol size is symbol area, square gives 'pleasing' image
plt.scatter(eqs['longitude'], eqs[' latitude'], color='red', marker='o', s=eqs[' magnitude']**2, transform=ccrs.Geodetic())

#transform coordinate system so can plot text adjacent to sysmbol
geodetic_transform = ccrs.Geodetic()._as_mpl_transform(ax)
text_transform = offset_copy(geodetic_transform, units='dots', x=0)

#fields, label by 2 letters
plt.text(ngatamariki_lonref, ngatamariki_latref, 'Ng', verticalalignment='center', horizontalalignment='center', transform=text_transform, fontsize=8)
plt.text(rotokawa_lonref, rotokawa_latref, 'Rk', verticalalignment='center', horizontalalignment='center', transform=text_transform, fontsize=8)
plt.text(mokai_lonref, mokai_latref, 'Mk', verticalalignment='center', horizontalalignment='center', transform=text_transform, fontsize=8)
plt.text(wairakei_tauhara_lonref, wairakei_tauhara_latref, 'Wk-Th', verticalalignment='center', horizontalalignment='center', transform=text_transform, fontsize=8)

plt.title('Earthquakes at Geothermal Fields Near Taupo, Since 2008', fontsize=8)

scale_bar(ax, ccrs.Mercator(), 5)

plt.savefig('taupo_geothermal_geonet_seismicity_map.png', dpi=400)

## Time Series

Separate section for each field, so can adjusted separately

### Rotokawa

In [ ]:
eqs['dist'] = haversine_np(eqs['longitude'],eqs[' latitude'],rotokawa_lonref,rotokawa_latref)

In [ ]:
eqsdist = eqs[eqs['dist']<3]

In [ ]:
eqsdist['dist'].describe()

In [ ]:
eqsdist['dist'].count()

In [ ]:
fig, (ax1) = plt.subplots(nrows=1,ncols=1, figsize=(15,5))

#magnitude vs time
eqsdist.plot(y=' magnitude', marker='o', markersize=5, linestyle='None', color='red', legend=False, ax=ax1, rot=0)

#yearly ticks and grid
majorTick = mpl.dates.YearLocator(1)
majorFormat = mpl.dates.DateFormatter('%Y')
ax1.xaxis.set_major_locator(majorTick)
ax1.xaxis.set_major_formatter(majorFormat)

ax1.set_ylim([0,4.2])
now = datetime.datetime.now()
ax1.set_xlim(start, str(now))
ax1.set_ylabel('Magnitude')
ax1.tick_params(labelright = True)

#titles and text
ax1.text(x = 0, y =0.95, transform=fig.transFigure, s = 'Earthquakes Located by GeoNet at Rotokawa Geothermal Field', horizontalalignment='left', fontsize = 20, color = 'black');
ax1.text(x = 0, y = 0.90, transform=fig.transFigure, s = 'GeoNet, www.geonet.org.nz', fontsize = 14, color = 'darkgray')
ax1.text(x = 1, y = 0.9, transform=fig.transFigure, s = 'SOURCE: GEONET, EARTHQUAKE DATABASE', horizontalalignment='right', fontsize = 14, color = 'darkgray');

plt.xticks(ha='center') 
plt.xlabel("") #get rid of useless 'date' label
plt.tight_layout()
plt.savefig('rotokawa_time-series.png', dpi=200)

### Ngatamariki

In [ ]:
eqs['dist'] = haversine_np(eqs['longitude'],eqs[' latitude'],ngatamariki_lonref,ngatamariki_latref)

In [ ]:
eqsdist = eqs[eqs['dist']<3]

In [ ]:
eqsdist['dist'].count()

In [ ]:
fig, (ax1) = plt.subplots(nrows=1,ncols=1, figsize=(15,5))

#magnitude vs time
eqsdist.plot(y=' magnitude', marker='o', markersize=5, linestyle='None', color='red', legend=False, ax=ax1, rot=0)

#yearly ticks and grid
majorTick = mpl.dates.YearLocator(1)
majorFormat = mpl.dates.DateFormatter('%Y')
ax1.xaxis.set_major_locator(majorTick)
ax1.xaxis.set_major_formatter(majorFormat)

ax1.set_ylim([0,4.2])
now = datetime.datetime.now()
ax1.set_xlim(start, str(now))
ax1.set_ylabel('Magnitude')
ax1.tick_params(labelright = True)

#titles and text
ax1.text(x = 0, y =0.95, transform=fig.transFigure, s = 'Earthquakes Located by GeoNet at Ngatamariki Geothermal Field', horizontalalignment='left', fontsize = 20, color = 'black');
ax1.text(x = 0, y = 0.90, transform=fig.transFigure, s = 'GeoNet, www.geonet.org.nz', fontsize = 14, color = 'darkgray')
ax1.text(x = 1, y = 0.9, transform=fig.transFigure, s = 'SOURCE: GEONET, EARTHQUAKE DATABASE', horizontalalignment='right', fontsize = 14, color = 'darkgray');

plt.xticks(ha='center') 
plt.xlabel("") #get rid of useless 'date' label
plt.tight_layout()
plt.savefig('ngatamariki_time-series.png', dpi=200)

### Mokai

In [ ]:
eqs['dist'] = haversine_np(eqs['longitude'],eqs[' latitude'],mokai_lonref,mokai_latref)

In [ ]:
eqsdist = eqs[eqs['dist']<3]

In [ ]:
eqsdist['dist'].count()

In [ ]:
fig, (ax1) = plt.subplots(nrows=1,ncols=1, figsize=(15,5))

#magnitude vs time
eqsdist.plot(y=' magnitude', marker='o', markersize=5, linestyle='None', color='red', legend=False, ax=ax1, rot=0)

#yearly ticks and grid
majorTick = mpl.dates.YearLocator(1)
majorFormat = mpl.dates.DateFormatter('%Y')
ax1.xaxis.set_major_locator(majorTick)
ax1.xaxis.set_major_formatter(majorFormat)

ax1.set_ylim([0,4.2])
now = datetime.datetime.now()
ax1.set_xlim(start, str(now))
ax1.set_ylabel('Magnitude')
ax1.tick_params(labelright = True)

#titles and text
ax1.text(x = 0, y =0.95, transform=fig.transFigure, s = 'Earthquakes Located by GeoNet at Mokai Geothermal Field', horizontalalignment='left', fontsize = 20, color = 'black');
ax1.text(x = 0, y = 0.90, transform=fig.transFigure, s = 'GeoNet, www.geonet.org.nz', fontsize = 14, color = 'darkgray')
ax1.text(x = 1, y = 0.9, transform=fig.transFigure, s = 'SOURCE: GEONET, EARTHQUAKE DATABASE', horizontalalignment='right', fontsize = 14, color = 'darkgray');

plt.xticks(ha='center') 
plt.xlabel("") #get rid of useless 'date' label
plt.tight_layout()
plt.savefig('mokai_time-series.png', dpi=200)

### Wairakei-Tauhara

In [ ]:
eqs['dist'] = haversine_np(eqs['longitude'],eqs[' latitude'],wairakei_tauhara_lonref,wairakei_tauhara_latref)

In [ ]:
eqsdist = eqs[eqs['dist']<8]

In [ ]:
eqsdist['dist'].count()

In [ ]:
fig, (ax1) = plt.subplots(nrows=1,ncols=1, figsize=(15,5))

#magnitude vs time
eqsdist.plot(y=' magnitude', marker='o', markersize=5, linestyle='None', color='red', legend=False, ax=ax1, rot=0)

#yearly ticks and grid
majorTick = mpl.dates.YearLocator(1)
majorFormat = mpl.dates.DateFormatter('%Y')
ax1.xaxis.set_major_locator(majorTick)
ax1.xaxis.set_major_formatter(majorFormat)

ax1.set_ylim([0,4.2])
now = datetime.datetime.now()
ax1.set_xlim(start, str(now))
ax1.set_ylabel('Magnitude')
ax1.tick_params(labelright = True)

#titles and text
ax1.text(x = 0, y =0.95, transform=fig.transFigure, s = 'Earthquakes Located by GeoNet at Wairakei-Tauhara Geothermal Field', horizontalalignment='left', fontsize = 20, color = 'black');
ax1.text(x = 0, y = 0.90, transform=fig.transFigure, s = 'GeoNet, www.geonet.org.nz', fontsize = 14, color = 'darkgray')
ax1.text(x = 1, y = 0.9, transform=fig.transFigure, s = 'SOURCE: GEONET, EARTHQUAKE DATABASE', horizontalalignment='right', fontsize = 14, color = 'darkgray');

plt.xticks(ha='center') 
plt.xlabel("") #get rid of useless 'date' label
plt.tight_layout()
plt.savefig('wairakei_time-series.png', dpi=200)